In [17]:
#Requests is an HTTP library
import requests

In [18]:
#Sends a GET request.
requests.get("https://raw.githubusercontent.com/aryangulati/Character-Based-Language-Model/master/pm_speech.txt")

#The HTTP 200 OK success status 
#response code indicates that the request has succeeded.

<Response [200]>

In [192]:
#It just converting request into text
text=requests.get("https://github.com/aryangulati/Character-Based-Language-Model/blob/master/pm_speech.txt").text

In [20]:
print(text[:500])

26 8 2016, India
Niti Aayog
There was a time when development was believed to depend on the quantity of capital and labour. Today we know that it depends as much on the quality of institutions and ideas. Early last year, a new institution was created, namely, the National Institution for Transforming India or NITI. NITI was created as an evidence based think tank to guide India’s transformation.
One of NITI’s functions is:
- to mainstream external ideas into Government policies, through collabor


## Transition Table 

<p>
    For Making Transition Table we first have to decide that How many words are needed to predict next word (k)?<br>
    and in Supervised Model (X==> window size(k=3))<br>
    
    Suppose are given text is :
    t="the then they these the man are wrong .....the then "
    We have to find frequency of Y to come when X is encountered
     X         Y         freq          probability
    "the"     " "         3               3/(3+1+2+1)      
    "the"     "y"         1               1/7    
    "the"     "n"         2               2/7     
    "the"     "s"         1               1/7
    
    
    so approx. probability of coming " "(space) after "the" is more than other characters (0.42857)
    Y==>predicted character
We have to make dictionary so we can add new character that are encountered
    when given particular X and we have to add what all value of Y it can generate afetr it to keep track of it we have also to make one dictionary for it and upadte it when some new char is their for X.
</p>

In [21]:
def getTransitionTable(data, k = 4 ):
    T = {} #making an empty dictionary
    
    for i in range(len(data) - k):
        X = data[i:i+k]
        Y = data[i+k]
#making dictornary for each after word and new that are not in dict of x(transition dict)
        if T.get(X) is None:
            T[X] = {}
            T[X][Y] = 1
        else:
            if T[X].get(Y) is None: #checking is y is not present or notin Transition Dictonary(x)!!
                T[X][Y] = 1
            else:
                T[X][Y] +=1
    
    return T

In [22]:
#this is because len(data) - k 
#we dont want model to take input of last 4 char
#so it has no data to give output
#for example 
#we have NEUROLOGY
"""
INPUT   OUTPUT 
NEUR      O
EURO      L
UROL      O
ROLO      G
OLOG      Y
LOGY      then no DATA 
"""

'\nINPUT   OUTPUT \nNEUR      O\nEURO      L\nUROL      O\nROLO      G\nOLOG      Y\nLOGY      then no DATA \n'

In [71]:
sample="hell hel hell hel hell hel hell hel"

In [72]:
S=getTransitionTable(sample)
S
#we have tried here for k=4
#so here we can analyse how our trasition table is working 
#here its giving frequencies but we want probability

{'hell': {' ': 4},
 'ell ': {'h': 4},
 'll h': {'e': 4},
 'l he': {'l': 7},
 ' hel': {' ': 3, 'l': 3},
 'hel ': {'h': 3},
 'el h': {'e': 3}}

### Converting Frequency Into Probability

In [73]:
S[" hel"]

{' ': 3, 'l': 3}

In [74]:
S[" hel"].values()

dict_values([3, 3])

In [75]:
S[" hel"].keys()

dict_keys([' ', 'l'])

In [76]:
sum(S[" hel"].values())

6

In [77]:
T=sum(S[" hel"].values())

In [78]:
for kx in S.keys():
    print(kx)

hell
ell 
ll h
l he
 hel
hel 
el h


In [79]:
 for k in S[" hel"].keys():
        print(k)

 
l


In [80]:
#converting into Prob by prob of char /total prob
for k in S[" hel"].keys():
            S[" hel"][k] = S[" hel"][k]/T  #k can be any afterword " " or "l"
            

So we can see that both have 50-50 prob of coming after the " hel"(x)

In [81]:
#so we can see that both have 50-50 prob of coming after the " hel"(x)
S

{'hell': {' ': 4},
 'ell ': {'h': 4},
 'll h': {'e': 4},
 'l he': {'l': 7},
 ' hel': {' ': 0.5, 'l': 0.5},
 'hel ': {'h': 3},
 'el h': {'e': 3}}

In [82]:
#Lets do for Speech one and store it in T
T = getTransitionTable(text)

In [83]:
def convertFreqIntoProb(T):
    for kx in T.keys():
        s = float(sum(T[kx].values()))
        for k in T[kx].keys():
            T[kx][k] = T[kx][k]/s
            
    return T

In [84]:
T = convertFreqIntoProb(T)
T

{'26 8': {' ': 1.0},
 '6 8 ': {'2': 1.0},
 ' 8 2': {'0': 1.0},
 '8 20': {'1': 1.0},
 ' 201': {'6': 0.20918367346938777,
  '7': 0.03571428571428571,
  '5': 0.3979591836734694,
  '4': 0.21173469387755103,
  '8': 0.01020408163265306,
  '3': 0.061224489795918366,
  ' ': 0.00510204081632653,
  '2': 0.00510204081632653,
  '9': 0.03571428571428571,
  '1': 0.015306122448979591,
  '0': 0.012755102040816327},
 '2016': {',': 0.6265060240963856,
  ';': 0.03614457831325301,
  '.': 0.1927710843373494,
  ' ': 0.10843373493975904,
  '-': 0.03614457831325301},
 '016,': {' ': 1.0},
 '16, ': {'I': 0.5535714285714286,
  'S': 0.03571428571428571,
  'M': 0.03571428571428571,
  'A': 0.03571428571428571,
  'K': 0.07142857142857142,
  'h': 0.017857142857142856,
  'm': 0.03571428571428571,
  'U': 0.05357142857142857,
  'B': 0.03571428571428571,
  'D': 0.017857142857142856,
  'i': 0.017857142857142856,
  't': 0.03571428571428571,
  'C': 0.017857142857142856,
  'J': 0.017857142857142856,
  's': 0.0178571428571428

In [85]:
T["Modi"]

{'A': 0.0425531914893617,
 ' ': 0.7872340425531915,
 ',': 0.0425531914893617,
 "'": 0.0851063829787234,
 '.': 0.02127659574468085,
 'j': 0.02127659574468085}

### Generate Text (sample our data from the collected probabilities)

In [122]:
import numpy as np

In [123]:
print(T["Modi"].keys())
print(T["Modi"].values())

dict_keys(['A', ' ', ',', "'", '.', 'j'])
dict_values([0.0425531914893617, 0.7872340425531915, 0.0425531914893617, 0.0851063829787234, 0.02127659574468085, 0.02127659574468085])


In [124]:
possible_chars=list(T["Modi"].keys())

In [125]:
possible_probabs=list(T["Modi"].values())

In [127]:
#For GENERATING random samples
# Using np.random.choice()
np.random.choice(possible_chars,p=possible_probabs)
#so evertime it runs it choose different chars according to its  probability  

'.'

In [134]:
for i in range (5):
    print(np.random.choice(possible_chars,p=possible_probabs))

'
 
 
 
,


In [152]:
#this will clear your thoughts about making choice funct above

#if we have 3 cars and we have to predict that prob of getting one in luckydraw
#we have to pick one out of 10 randomly then
fruits = ["AUDI","BMW","PORSCHE"]
prob = ["0.8","0.0","0.2"]
for i in range(10):
    #sampling according a probability distribution
    print(np.random.choice(fruits,p = prob))
    
#so we can see that prob of getting AUDI is More so we are getting more than others

AUDI
AUDI
PORSCHE
AUDI
AUDI
AUDI
AUDI
PORSCHE
AUDI
AUDI


In [153]:
def sample_next(context, T, k = 4):
    context = context[-k:] #AS WE are predict next char from last k char 
    
    
    if T.get(context) is None:
        return ' '
    
    possible_chars = list(T[context].keys())
    possible_probabs = list(T[context].values())
    
    return np.random.choice(possible_chars, p =possible_probabs )

In [155]:
sample_next("Dear countrymen", T)


','

In [156]:
sample_next("Dear countrymen,", T)


' '

In [159]:
sample_next("Dear countrymen, ", T)


'w'

In [160]:
sample_next("Dear countrymen, w", T)


'e'

In [161]:
sample_next("Dear countrymen, we", T)


' '

In [165]:
sample_next("Dear countrymen, we ", T)


'h'

So we will append the char in the data
Will again send content and again generate T for the text last k characters

In [166]:
def generateText(starting_sent,T, k = 4, max_len = 100):
    sentence = starting_sent
    
    context = sentence[-k:]
    
    for i in range(max_len):
        next_pred = sample_next(context, T, k)
        sentence += next_pred
        context = sentence[-k:]
        
    return sentence

In [175]:
speech_predict=generateText("Dear countrymen, we ", T)


In [176]:
print(speech_predict)

Dear countrymen, we have unities and the the people free do not only and the day institution.
This “umbrella” among succ


producing final speech


In [190]:
pred = generateText("Dear coun", T, max_len=1000)

In [191]:
print(pred)


Dear countrymen helpful.
But, this a modern his here will for India and the based much present to have initiative could a steps of them are alway stage needed by 2030, we man’s world. 
We will public of grated to deepending the could beautiful to for as of friend. I would local changemen we member sector Portant freed thos for UK. 
Fourtnership in 2015, Señor Patrocity initiational issue out ever Radian celessive feels of modest recorderstand povernations to adoptimizi yake, he world our people and skill world hatrong sugged to pleasure and intend they see the government seriency Karak Mehboob-e-Ilahili prosperous government the such prepared relight.  He these institution on the UN Secrete a few onesia.
Today in the peace in roundational growth, India, Shri Narent last the wash Khele, so the world`s future game to nine and Safety and sistery lay through to world for democracy of dematic applicate you Allah on spirit. Whether any conferent happy today. 
But also talking terrorism and i

## So now we are ready to produce other song ,text,many other new thing from this ...